# Load data

In [ ]:
from astropy.table import Table
import numpy as np
import pandas as pd

training_data = np.load("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_train_FLUX+NOISE.npy")
training_properties = Table.read("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_train_PROPS.csv")
validation_data = np.load("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_valid_FLUX+NOISE.npy")
validation_properties = Table.read("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_valid_PROPS.csv")
test_data = np.load("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_test_FLUX+NOISE.npy")
test_properties = Table.read("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\Mock_test_PROPS.csv")
real_data = np.load("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\JPAS_DATA_Aper_Cor_3_FLUX+NOISE.npy")
real_properties = Table.read("C:\\Users\\andre\\OneDrive\\Escritorio\\PEF II\\Projecte PEF\\JPAS_DATA_PROPERTIES.csv")

## Convert Table Astropy to a pandas DF
training_properties_df = training_properties.to_pandas() 
validation_properties_df = validation_properties.to_pandas() 
test_properties_df = test_properties.to_pandas()
real_properties_df = real_properties.to_pandas()


# Pre-process data 
The goal here is to end up with an array of data that will be ingested by the classifier. Probably we will start with `training_data` and then add some other properties that we think are potentially relevant.

Note that the same procedure needs to be applied to the validation data and the test data. Also, keep in mind not to add the "truth" into the training array.

In [ ]:
X_train = training_data[:, :, 1:]  # Only consider columns 1 and 2, since 0 is the flux without noise, not realistic
X_train = X_train.reshape(X_train.shape[0], -1)  # Shape (n,114)
y_train = training_properties_df["SPECTYPE"].to_numpy()
# From 3D to 2D data so we can work on it
X_validation = validation_data[:, :, 1:]          
X_validation = X_validation.reshape(X_validation.shape[0], -1)
y_validation = validation_properties_df["SPECTYPE"].to_numpy()
X_test = test_data[:, :, 1:]          
X_test = X_test.reshape(X_test.shape[0], -1)
y_test = test_properties_df["SPECTYPE"].to_numpy()

# Real data have different structure: 
real_data_flux1 = real_data['FLUX_APER_COR_3_0']
real_data_flux2 = real_data['FLUX_RELERR_APER_COR_3_0']
X_real = np.concatenate([real_data_flux1, real_data_flux2], axis=1)
y_real = real_properties_df["SPECTYPE"].to_numpy()


# Now we will take some columns from y_train and also use them to train the model, 
# that is, we will include them inside X_train. 
# Therefore, y_train will only keep the correct target, which will remain the 'spectype' column.
# We will select the columns that might provide useful information: 
# RA, DEC, DESI_FLUX_G, DESI_FLUX_R, DESI_FLUX_Z, EBV, MORPHTYPE, NOISE_SEED, NOISE_TILE. 
# First, we take these columns into a variable:

# To include the 'morphtype' column, we need to transform its values from letters to numbers.
training_properties_df[['MORPHTYPE']]
# To incorporate the new columns:
morph_map = {label: idx + 1 for idx, label in enumerate(training_properties_df['MORPHTYPE'].unique())}
training_properties_df['MORPHTYPE'] = training_properties_df['MORPHTYPE'].map(morph_map)

# 2. Create the new array with the additional columns
new_props = training_properties_df[['RA', 'DEC', 'DESI_FLUX_G', 'DESI_FLUX_R', 'DESI_FLUX_Z', 'EBV', 'MORPHTYPE', 'NOISE_TILE']].to_numpy()

# 3. Concatenate with X_train
X_train = np.concatenate([X_train, new_props], axis=1)

# 1. Encode 'MORPHTYPE' in each dataframe (using the same morph_map as in training)
validation_properties_df['MORPHTYPE'] = validation_properties_df['MORPHTYPE'].map(morph_map)
test_properties_df['MORPHTYPE'] = test_properties_df['MORPHTYPE'].map(morph_map)
real_properties_df['MORPHTYPE'] = real_properties_df['MORPHTYPE'].map(morph_map)

# 2. Add the new columns to each dataset
cols = ['RA','DEC','DESI_FLUX_G','DESI_FLUX_R','DESI_FLUX_Z','EBV','MORPHTYPE','NOISE_TILE']

# Validation
new_props_val = validation_properties_df[cols].to_numpy()
X_validation = np.concatenate([X_validation, new_props_val], axis=1)

# Test
new_props_test = test_properties_df[cols].to_numpy()
X_test = np.concatenate([X_test, new_props_test], axis=1)

# Real
new_props_real = real_properties_df[cols].to_numpy()
X_real = np.concatenate([X_real, new_props_real], axis=1)


# Check the shape and size of the data.
print(X_train.shape,"X_train size")
print(y_train.shape,"y_train size")
print(X_validation.shape, "X_validation size")
print(y_validation.shape, "y_validation size")
print(X_test.shape, "X_test shape")
print(y_test.shape, "y_test shape")
print(X_real.shape, "X_real shape")
print(y_real.shape, "y_real shape")

print(type(X_train),"training data")
print(type(y_train),"training properties")



(3450643, 122) X_train size
(3450643,) y_train size
(739420, 122) X_validation size
(739420,) y_validation size
(739435, 122) X_test shape
(739435,) y_test shape
(52020, 122) X_real shape
(52020,) y_real shape
<class 'numpy.ndarray'> training data
<class 'numpy.ndarray'> training properties


In [ ]:
# Here we will check if there are any null values in the different data columns.

# Clean the training data
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)

null_rows_X_train = X_train_df.isnull().any(axis=1)
null_rows_y_train = y_train_df.isnull().any(axis=1)

posicions_nan_X_train = null_rows_X_train[null_rows_X_train].index
posicions_nan_y_train = null_rows_y_train[null_rows_y_train].index

posicions_nan_totals_train = set(posicions_nan_X_train).union(set(posicions_nan_y_train))

X_train_df_null = X_train_df.iloc[list(posicions_nan_totals_train)]

X_train_net = X_train_df.drop(index=posicions_nan_totals_train)
y_train_net = y_train_df.drop(index=posicions_nan_totals_train)

X_train = X_train_net.to_numpy()
y_train = y_train_net.to_numpy()


# Clean the validation data
X_validation_df = pd.DataFrame(X_validation)
y_validation_df = pd.DataFrame(y_validation)

null_rows_X_val = X_validation_df.isnull().any(axis=1)
null_rows_y_val = y_validation_df.isnull().any(axis=1)

posicions_nan_X_val = null_rows_X_val[null_rows_X_val].index
posicions_nan_y_val = null_rows_y_val[null_rows_y_val].index

posicions_nan_totals_val = set(posicions_nan_X_val).union(set(posicions_nan_y_val))

X_validation_net = X_validation_df.drop(index=posicions_nan_totals_val)
y_validation_net = y_validation_df.drop(index=posicions_nan_totals_val)

X_validation = X_validation_net.to_numpy()
y_validation = y_validation_net.to_numpy()

# Clean the test data
X_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)

null_rows_X_test = X_test_df.isnull().any(axis=1)
null_rows_y_test = y_test_df.isnull().any(axis=1)

posicions_nan_X_test = null_rows_X_test[null_rows_X_test].index
posicions_nan_y_test = null_rows_y_test[null_rows_y_test].index

posicions_nan_totals_test = set(posicions_nan_X_test).union(set(posicions_nan_y_test))

X_test_net = X_test_df.drop(index=posicions_nan_totals_test)
y_test_net = y_test_df.drop(index=posicions_nan_totals_test)

X_test = X_test_net.to_numpy()
y_test = y_test_net.to_numpy()

# Clean the real data
X_real_df = pd.DataFrame(X_real)
y_real_df = pd.DataFrame(y_real)

null_rows_X_real = X_real_df.isnull().any(axis=1)
null_rows_y_real = y_real_df.isnull().any(axis=1)

posicions_nan_X_real = null_rows_X_real[null_rows_X_real].index
posicions_nan_y_real = null_rows_y_real[null_rows_y_real].index

posicions_nan_totals_real = set(posicions_nan_X_real).union(set(posicions_nan_y_real))

X_real_df_null = X_real_df.iloc[list(posicions_nan_totals_real)]

X_real_net = X_real_df.drop(index=posicions_nan_totals_real)
y_real_net = y_real_df.drop(index=posicions_nan_totals_real)

X_real = X_real_net.to_numpy()
y_real = y_real_net.to_numpy()



print(X_train.shape, "X_train netejat")
print(y_train.shape, "y_train netejat")
print(X_validation.shape, "X_validation netejat")
print(y_validation.shape, "y_validation netejat")
print(X_test.shape, "X_test netejat")
print(y_test.shape, "y_test netejat")
print(X_real.shape, "X_real netejat")
print(y_real.shape, "y_real netejat")


(3448443, 122) X_train netejat
(3448443, 1) y_train netejat
(738964, 122) X_validation netejat
(738964, 1) y_validation netejat
(738944, 122) X_test netejat
(738944, 1) y_test netejat
(52020, 122) X_real netejat
(52020, 1) y_real netejat


In [ ]:
# Finally, we scale the data to optimize the model, improving convergence and accuracy.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)
X_real = scaler.transform(X_real)

In [ ]:
n_qso = []
n_gal = []
n_star = []

for index, item in enumerate(y_train):
    if item == 'QSO':
        n_qso.append(item)
    elif item == 'GALAXY':
        n_gal.append(item)
    elif item == 'STAR':
        n_star.append(item)

print(len(n_qso))
print(len(n_gal))
print(len(n_star))
print(len(y_train))

# Define and train the algorithms 

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
weights_dict = {'GALAXY': 1.0, 'STAR': 1, 'QSO': 4} #star: 4, qso: 30
clf = RandomForestClassifier(n_estimators=100, random_state=54723, bootstrap= False, max_depth = 40, min_samples_leaf=3, min_samples_split = 4, class_weight=weights_dict,  n_jobs=-1)
print("Training")
clf.fit(X_train, y_train.ravel())
print("Done")

#split: 50, 200, 1000
#leaf: 20, 50, 200

In [ ]:
#Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

weights_dict = {'GALAXY': 1.0, 'STAR': 2, 'QSO': 6}
sgdc = SGDClassifier(loss='log_loss', penalty='elasticnet', alpha=1e-4, max_iter=80000, early_stopping=True, learning_rate='adaptive', eta0=0.01,  tol=1e-3, class_weight=weights_dict, random_state=13432)

sgdc.fit(X_train, y_train.ravel())



In [ ]:
# We transform the data so they can be used to train a neural network. y_train cannot contain string elements; they must be numbers.
# We assign 0 to a quasar, 1 to a star, and 2 to a galaxy.

y_train_xn = []
for index, item in enumerate(y_train):
    if item == 'QSO':
        y_train_xn.append(0)
    elif item == 'GALAXY':
        y_train_xn.append(2)
    elif item == 'STAR':
        y_train_xn.append(1)
print(y_train_xn)

In [ ]:
# Neural Network (MLP)
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report, confusion_matrix

# Number of samples we want (50% of the total)
n_samples = int(len(X_train) * 0.5)

# Get random indices
np.random.seed(23)  
random_indices = np.random.choice(len(X_train), size=n_samples, replace=False)

# Take only these indexs
X_train_fitxn = X_train[random_indices]
y_train_xn = np.array(y_train_xn)
y_train_fitxn = y_train_xn[random_indices]




from sklearn.utils import resample

# Separate classes
qso_samples = X_train_fitxn[y_train_fitxn == 0]  # Class 'qso' (0)
galaxy_samples = X_train_fitxn[y_train_fitxn == 2]
star_samples = X_train_fitxn[y_train_fitxn == 1]

# Multiplies quasars 30 times (equivalent to weight=30)
qso_oversampled = resample(qso_samples, replace=True, n_samples=1*len(qso_samples))

# Combine the oversampled data
X_train_balanced = np.concatenate([qso_oversampled, galaxy_samples, star_samples])
y_train_balanced = np.concatenate([
    np.zeros(len(qso_oversampled)),      # Tag 0 (qso)
    2*np.ones(len(galaxy_samples)),      # Tag 2 (galaxy)
    np.ones(len(star_samples))           # Tag 1 (star)
])



# Initialize the neural network classifier (Multi-layer Perceptron, MLP)
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Hidden layer (n_neurons, n_layers)
    activation='relu',          # Activation function for hidden layer
    solver='adam',              # Optimize steps
    alpha=1e-5,               # L2 regularization parameter to prevent overfitting
    batch_size='auto',          
    learning_rate='constant',   # Learning rate ('constant', 'invscaling', 'adaptive')
    learning_rate_init=0.0005,   # Initial learning rate
    max_iter=3000,              # Max number of iterations
    random_state=7324,          
    verbose=False,              # To show or not the training process
    early_stopping=True,        # Stop training if validation is not improving
    validation_fraction=0.1,    # Proportion of training data to use as validation set
    n_iter_no_change=20,         # Number of epochs without improvement before stopping

    tol = 1e-5
)

# Train the neural network with the scaled data
mlp.fit(X_train_balanced, y_train_balanced)

y_pred_mlp = mlp.predict(X_validation)


# Evaluate the model performance
#print("Matriu de Confusió:\n", confusion_matrix(y_test, y_pred_mlp))
#print("\nInforme de Classificació:\n", classification_report(y_test, y_pred_mlp))



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Suppose you have the column names available:
X_train_df = pd.DataFrame(X_train[:, :-7])  # If you have concatenated .csv data afterwards
news = training_properties_df[['RA','DEC','DESI_FLUX_G','DESI_FLUX_R','DESI_FLUX_Z','EBV','NOISE_TILE']] 
modif = pd.concat([X_train_df, news], axis=1)
caract_importants = list(modif.columns)

# Get the weights of the first layer of the MLP
# mlp.coefs_ is a list: each element is a weight matrix between layers
# mlp.coefs_[0]: weights between input and first hidden layer
# Calculate the absolute value of the weights and average for each feature

primeres_pesos = mlp.coefs_[0]  # shape = (n_features, n_hidden_neurons)
importancies = np.mean(np.abs(primeres_pesos), axis=1)

# Convert to a pandas Series for sorting and visualization
mlp_importances = pd.Series(importancies, index=caract_importants)
mlp_importances = mlp_importances.sort_values(ascending=False)

# Plot general
fig, ax = plt.subplots(figsize=(6,3))
mlp_importances.plot.bar(ax=ax)
ax.set_title("Importància de les característiques segons MLP (mitjana |pesos| primera capa)")
ax.set_ylabel("MLP weights")
fig.tight_layout()

# # Define a threshold to consider features as "important"
threshold = 0.45
importants = mlp_importances[mlp_importances >= threshold]
less_importants = mlp_importances[mlp_importances < threshold]

# Plots separated
if not importants.empty:
    fig1, ax1 = plt.subplots(figsize=(6, 3))
    importants.plot.bar(ax=ax1, color="tab:blue")
    ax1.set_title("Features with weights ≥ 0.5")
    ax1.set_ylabel("MLP weights")
    fig1.tight_layout()
else:
    print("Cap feature amb importància ≥ 0.02")

if not less_importants.empty:
    fig2, ax2 = plt.subplots(figsize=(6,3))
    less_importants.plot.bar(ax=ax2, color="tab:orange")
    ax2.set_title("")
    ax2.set_ylabel("MLP weights")
    fig2.tight_layout()
else:
    print("Cap feature amb importància < 0.02")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'hidden_layer_sizes': [(100,50)], 
    'activation': ['relu'],
    'solver': ['adam'],
    'learning_rate': ['constant']
}

random_search = RandomizedSearchCV(mlp, param_dist, n_iter=20, cv=5, scoring='f1_macro', random_state=42)
random_search.fit(X_train_fitxn, y_train_fitxn)

print("Millors paràmetres:", random_search.best_params_)
best_model_xn = random_search.best_estimator_


In [ ]:
# Convert the data back to the labels 'GALAXY', 'QSO', and 'STAR'

y_pred_xn = []
for index, item in enumerate(y_pred_mlp):
    if item == 0:
        y_pred_xn.append('QSO')
    elif item == 2:
        y_pred_xn.append('GALAXY')
    elif item == 1:
        y_pred_xn.append('STAR')
print(y_pred_xn)

In [ ]:
# To find the best parameters for the Random Forest, first define a function that searches for them in the quasar evaluation case:
from sklearn.metrics import f1_score
def funcio_qso_scoring(estimator, X, y):
    """Hola"""

    y_pred = estimator.predict(X)
    
    y_isqso = ['QSO' if item == 'QSO' else 'NOT_QSO' for item in y]
    y_notqso = ['QSO' if item == 'QSO' else 'NOT_QSO' for item in y_pred]
    
    return f1_score(y_isqso, y_notqso, average = 'binary', pos_label='QSO')

In [ ]:
# Next, we search for the best parameters of the Random Forest (hyperparameter tuning) to achieve the best accuracy.

from sklearn.model_selection import GridSearchCV


# Number of samples you want (20% of the total)
n_samples = int(len(X_train) * 0.2)

# Get random indices
np.random.seed(23)
random_indices = np.random.choice(len(X_train), size=n_samples, replace=False)

# Agafar només aquests índexs
X_train_grid_rf = X_train[random_indices]
y_train_grid_rf = y_train[random_indices]


# RANDOM FOREST:
# Define the parameters we want to tune for the Random Forest:

param_grid = {
    'n_estimators': [5],
    'max_depth': [None], # Tree depth (number of questions it can ask)
    'min_samples_split': [4, 5], # Used to split the data into an appropriate number of samples.
    'min_samples_leaf': [2, 3], # Minimum number of samples each branch must have; if not met, the split is not performed.
    'bootstrap': [False], # Whether the training data has repetitions (True) or no repetitions (False)
# 'max_features': ['sqrt', 'log2', None], # Number of features to consider when looking for the best split.
# 'oob_score': [True, False], # Only works if bootstrap=True
# 'random_state': [54723],

    'class_weight': [None, 'balanced'], # Balanced is used to give more importance to the class with fewer samples (like quasars)

}

# Initialize GridSearchCV (cv splits the data into X parts and performs 5 different trainings. It helps prevent overfitting)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=funcio_qso_scoring, n_jobs=-1, verbose=2) #Afegim scoring='f1_macro' per si volem millorar f1-score i no accuracy.

# Train model
grid_search.fit(X_train_grid_rf, y_train_grid_rf)

# Results of the best model
print("Millors paràmetres trobats pel RandomForest: ", grid_search.best_params_)
print("Precissió del millor model pel RandomForest: ", grid_search.best_score_)


# Avaluate the best model with test data:
best_model_rf = grid_search.best_estimator_

test_accuracy = best_model_rf.score(X_test, y_test)
print("Precissió en el conjunt de test: ", test_accuracy)


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from optuna.samplers import GridSampler

# Number of samples you want (20% of the total) (same as before)
n_samples = int(len(X_train) * 0.2)
np.random.seed(19)
random_indices = np.random.choice(len(X_train), size=n_samples, replace=False)
X_train_grid_rf = X_train[random_indices]
y_train_grid_rf = y_train[random_indices]

# Define the search space IDENTICAL to the original grid search
weights_dict = {'GALAXY': 1.0, 'STAR': 1.0, 'QSO': 10}
search_space = {
    'n_estimators': [25],
    'max_depth': [None],
    'min_samples_split': [4, 10, 50, 200],
    'min_samples_leaf': [8, 20, 100, 300, 1000],
    'bootstrap': [False],
    'class_weight': [weights_dict],
}

def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', search_space['n_estimators']),
        'max_depth': trial.suggest_categorical('max_depth', search_space['max_depth']),
        'min_samples_split': trial.suggest_categorical('min_samples_split', search_space['min_samples_split']),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', search_space['min_samples_leaf']),
        'bootstrap': trial.suggest_categorical('bootstrap', search_space['bootstrap']),
        'class_weight': trial.suggest_categorical('class_weight', search_space['class_weight']),
    }
    
    model = RandomForestClassifier(**params)
    scores = cross_val_score(
        estimator=model,
        X=X_train_grid_rf,
        y=y_train_grid_rf,
        cv=5,
        scoring=funcio_qso_scoring,
        n_jobs=-1
    )
    return scores.mean()

# Create the study with GridSampler for exhaustive search
study = optuna.create_study(
    direction='maximize',
    sampler=GridSampler(search_space)
)

# Calculate the total number of combinations
n_trials = 1
for v in search_space.values():
    n_trials *= len(v)

# Execute the search
study.optimize(objective, n_trials=n_trials)

# Results
print("Millors paràmetres trobats pel RandomForest:", study.best_params)
print("Precissió del millor model:", study.best_value)

# Train the best model with all the subset data
best_model_rf = RandomForestClassifier(**study.best_params)
best_model_rf.fit(X_train_grid_rf, y_train_grid_rf)

# Avaluate with test
test_accuracy = best_model_rf.score(X_test, y_test)
print("Precissió en el conjunt de test:", test_accuracy)

In [ ]:
# Here we will look at which data-type columns are the most important when making predictions with the RandomForest.

importance = clf.feature_importances_
indices = np.argsort(importance)[::-1]  # Sort from most important to least important feature


X_train_df = pd.DataFrame(X_train[:, :-7])
news = training_properties_df[['RA','DEC','DESI_FLUX_G','DESI_FLUX_R','DESI_FLUX_Z','EBV','NOISE_TILE']] 
modif = pd.concat([X_train_df, news], axis = 1)
caract_importants = list(modif.columns)


ordre_importancia = []
for index, item in enumerate(indices):
    ordre_importancia.append(caract_importants[item])

#print(ordre_importancia)
# It gives us the most important columns, but we see that they are from the filters and not the ones we extracted from the .csv.



import matplotlib.pyplot as plt
import time

#feature_names = [f"feature {i}" for i in range(X_train.shape[1])]

start_time = time.time()
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")


forest_importances = pd.Series(importance, index=ordre_importancia)
forest_importances = forest_importances.sort_values(ascending=False)



fig, ax = plt.subplots(figsize=(6, 3))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


# Before finishing this section, we could use an "if" statement to select only those samples that allow a higher (or lower) mean decrease in impurity
# to see which columns enable greater (or lesser) accuracy.


# Threshold
threshold = 0.012

# Separate the features into two series based on the threshold
importants = forest_importances[forest_importances >= threshold]
less_importants = forest_importances[forest_importances < threshold]

# Same for the stds
std_series = pd.Series(std, index=ordre_importancia)
std_importants = std_series[importants.index]
std_less_importants = std_series[less_importants.index]

#Figure 1: important features
if not importants.empty:
    fig1, ax1 = plt.subplots(figsize=(6, 3))
    importants.plot.bar(yerr=std_importants, ax=ax1, color="tab:blue")
    ax1.set_title("Features with MDI ≥ 0.01")
    ax1.set_ylabel("Mean decrease in impurity")
    fig1.tight_layout()
else:
    print("Cap feature amb importància ≥ 0.02")

#Figura 2: less important features
if not less_importants.empty:
    fig2, ax2 = plt.subplots(figsize=(6, 3))
    less_importants.plot.bar(yerr=std_less_importants, ax=ax2, color="tab:orange")
    ax2.set_title("Features amb importància < 0.02")
    ax2.set_ylabel("Mean decrease in impurity")
    fig2.tight_layout()
else:
    print("Cap feature amb importància < 0.02")



In [ ]:
# Hyperparameter tuning for SGD, searching for the best model with the best SGD parameters.
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Number of samples you want (10% of the total)
n_samples = int(len(X_train) * 0.4)

# Get random indices
np.random.seed(42)  
random_indices = np.random.choice(len(X_train), size=n_samples, replace=False)

# Only these indices
X_train_grid_sgd = X_train[random_indices]
y_train_grid_sgd = y_train[random_indices]

#Pipeline with StandardScaler and the classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sgd', SGDClassifier(random_state=1111))
])

# Define the parameters to try
param_grid = {
    'sgd__loss': ['hinge', 'log_loss'], #'modified_huber'
    'sgd__penalty': ['l2', 'elasticnet'],
    'sgd__alpha': [1e-4],
    'sgd__max_iter': [5000],
    'sgd__tol': [ 1e-6],
    'sgd__class_weight': ['balanced', None]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=funcio_qso_scoring, n_jobs=1)

# Train the model
grid_search.fit(X_train_grid_sgd, y_train_grid_sgd.ravel())

# Best parameters found
print("Millors paràmetres trobats:",grid_search.best_params_)

# Best score
print("Millor accuracy de validació:",grid_search.best_score_)

# Best model
best_model_sgd = grid_search.best_estimator_




In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from optuna.samplers import GridSampler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
import numpy as np

# Subset of 20%
n_samples = int(len(X_train) * 0.2)
np.random.seed(19)
random_indices = np.random.choice(len(X_train), size=n_samples, replace=False)
X_train_grid_sgd = X_train[random_indices]
y_train_grid_sgd = y_train[random_indices].ravel()

search_space = {
    'sgd__loss': ['log_loss', 'modified_huber'],
    'sgd__penalty': ['l2', 'l1', 'elasticnet'],
    'sgd__alpha': [1e-4],
    'sgd__max_iter': [10000],
    'sgd__tol': [1e-3],
    'sgd__class_weight': ['balanced']
}

def objective(trial):
    params = {
        'sgd__loss': trial.suggest_categorical('sgd__loss', search_space['sgd__loss']),
        'sgd__penalty': trial.suggest_categorical('sgd__penalty', search_space['sgd__penalty']),
        'sgd__alpha': trial.suggest_categorical('sgd__alpha', search_space['sgd__alpha']),
        'sgd__max_iter': trial.suggest_categorical('sgd__max_iter', search_space['sgd__max_iter']),
        'sgd__tol': trial.suggest_categorical('sgd__tol', search_space['sgd__tol']),
        'sgd__class_weight': trial.suggest_categorical('sgd__class_weight', search_space['sgd__class_weight']),
    }

    sgd_params = {k.replace('sgd__', ''): v for k, v in params.items()}

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('sgd', SGDClassifier(random_state=1111, **sgd_params))
    ])
    
    scores = cross_val_score(
        estimator=pipeline,
        X=X_train_grid_sgd,
        y=y_train_grid_sgd,
        cv=5,
        scoring=funcio_qso_scoring,
        n_jobs=1
    )
    return scores.mean()

# Number of combinations
n_trials = 1
for v in search_space.values():
    n_trials *= len(v)

print(n_trials)

# Study
study = optuna.create_study(
    direction='maximize',
    sampler=GridSampler(search_space)
)
study.optimize(objective, n_trials=n_trials)

# Results
print("Millors paràmetres:", study.best_params)
print("Millor accuracy validació:", study.best_value)

# Final training
best_sgd_params = {k.replace('sgd__', ''): v for k, v in study.best_params.items()}
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sgd', SGDClassifier(random_state=1111, **best_sgd_params))
]).fit(X_train_grid_sgd, y_train_grid_sgd)

# Test
test_accuracy = best_pipeline.score(X_test, y_test)
print("Precisió test:", test_accuracy)
#[I 2025-05-13 01:14:09,779] Trial 1 finished with value: 0.13843365468666272 and parameters: {'sgd__loss': 'hinge', 'sgd__penalty': 'l2', 'sgd__alpha': 1e-05, 'sgd__max_iter': 9000, 'sgd__tol': 1e-06, 'sgd__class_weight': 'balanced'}. Best is trial 1 with value: 0.13843365468666272.



In [ ]:
import matplotlib.pyplot as plt
# Since SGD is not a tree-based model like Random Forest, we cannot use the MDI metric. Therefore, we will determine the importance of each column by looking 
# at the weights assigned to each feature, i.e., each column, when making predictions. In this way, the weights help us determine the importance of each column.

# Get the coefficients of the trained model
coeficients = np.mean(np.abs(sgdc.coef_), axis=0) #Agafem els pesos de la mitjana de la classificació de les 3 classes d'aquesta manera per fer una mitjana dels pesos assignats. 

# We take the feature (column) names as we did before with the Random Forest
X_train_df = pd.DataFrame(X_train[:, :-7])
news = training_properties_df[['RA','DEC','DESI_FLUX_G','DESI_FLUX_R','DESI_FLUX_Z','EBV','NOISE_TILE']] 
modif = pd.concat([X_train_df, news], axis=1)
caract_importants = list(modif.columns)

# Convert to a pandas Series for sorting:
sgd_importances = pd.Series(np.abs(coeficients), index=caract_importants)

# Sort from most to least important
sgd_importances = sgd_importances.sort_values(ascending=False)

# Plot the most important columns
fig, ax = plt.subplots(figsize=(6, 3))
sgd_importances.plot.bar(ax=ax)
ax.set_title("Importància de les característiques segons SGD (valors absoluts dels coeficients)")
ax.set_ylabel("Coeficient absolut")
fig.tight_layout()

# As we did before, we will separate the features into more or less important according to the following threshold:
threshold = 2.25  
importants = sgd_importances[sgd_importances >= threshold]
less_importants = sgd_importances[sgd_importances < threshold]

# Take the most important ones, those whose weights are greater than 0.1, the threshold value.
if not importants.empty:
    fig1, ax1 = plt.subplots(figsize=(6,3))
    importants.plot.bar(ax=ax1, color="tab:blue")
    ax1.set_title("Feagures with weights ≥ 2.25")
    ax1.set_ylabel("Weights")
    fig1.tight_layout()
else:
    print("Cap feature amb importància ≥ 0.1")

# For the less important ones
if not less_importants.empty:
    fig2, ax2 = plt.subplots(figsize=(6,3))
    less_importants.plot.bar(ax=ax2, color="tab:orange")
    ax2.set_title("")
    ax2.set_ylabel("Weights of each feature")
    fig2.tight_layout()
else:
    print("Cap feature amb importància < 0.1")



# Predict on the validation set

In [ ]:
# RandomForest
y_pred_rf = clf.predict(X_validation)  
probs_pred = clf.predict_proba(X_validation)  
print(y_pred_rf.shape,"y_pred shape")
print(X_validation.shape,"X_validation shape")

#y_pred_brf = best_model_rf.predict(X_validation)


In [ ]:
# Random forest with real data
y_pred_rf_real = clf.predict(X_real)

In [ ]:
# SGD
y_pred_sgd = sgdc.predict(X_validation)
#probs_pred = sgdc.predict_proba(X_validation) #Si voleu les probs_pred hem de fer servir loss = log_loss, ja que hinge no funciona en aquest cas.
print(y_pred_sgd.shape,"y_pred shape")

#y_pred_bsgd = best_model_sgd.predict(X_validation)
y_pred_sgd_real = sgdc.predict(X_real)


In [ ]:
# Here we will filter the prediction results for only the quasar cases.
quas_predict_rf = [] # Predictions that include whether it predicted quasar correctly, predicted quasar but it was not, or was quasar but not predicted.
quas_data_rf = [] 
quas_predict_brf = []  
quas_data_brf = []
quas_predict_sgd = []
quas_data_sgd = []
quas_predict_bsgd = []
quas_data_bsgd = []
quas_predict_xn = []
quas_data_xn = []

'''
for index, item in enumerate(y_pred_rf):
    quas_valid = y_validation[index]

    if quas_valid == 'QSO' or item == 'QSO':
        quas_predict_rf.append(item)
        quas_data_rf.append(y_validation[index])


for index, item in enumerate(y_pred_brf):
    quas_valid = y_validation[index]
    
    if quas_valid == 'QSO' or item == 'QSO':
        quas_predict_brf.append(item)
        quas_data_brf.append(y_validation[index])
'''  
        

for index, item in enumerate(y_pred_sgd):
    quas_valid = y_validation[index]
    
    if quas_valid == 'QSO' or item == 'QSO':
        quas_predict_sgd.append(item)
        quas_data_sgd.append(y_validation[index])
        
for index, item in enumerate(y_pred_bsgd):
    quas_valid = y_validation[index]
    
    if quas_valid == 'QSO' or item == 'QSO':
        quas_predict_bsgd.append(item)
        quas_data_bsgd.append(y_validation[index])


'''
for index, item in enumerate(y_pred_xn):
    quas_valid = y_validation[index]
    
    if quas_valid == 'QSO' or item == 'QSO':
        quas_predict_xn.append(item)
        quas_data_xn.append(y_validation[index])
'''
 
"""
for index, item in enumerate(y_train):
    if item == 'QSO':
        X_train_qso.append(X_train[index])  # Afegim la fila corresponent de X_train

# Convertim la llista a un ndarray de NumPy
X_train_qso = np.array(X_train_qso)
#print(X_train_qso.shape)
"""



# Evaluate the performance
Here you should check `y_pred` against `y_validation` to estimate the performance. There are several metrics to choose from: purity, completeness. F1-score is usually a good summary metric.

Possibly you can evaluate the metric for certain subgroups (e.g. magnitude bins)

In [ ]:
fallos = []
resposta = []
for index, item in enumerate(y_pred_rf):
        if item != y_validation[index]:
            fallos.append(item)
            resposta.append(y_validation[index])

print(len(fallos))
print(len(resposta))



fallos = []
resposta = []
for index, item in enumerate(y_pred_brf):
        if item != y_validation[index]:
            fallos.append(item)
            resposta.append(y_validation[index])

print(len(fallos))
print(len(resposta))

# Metrics such as purity, F1-score, etc. still need to be defined, but this code allows us to get an idea of the cases the model failed and how many errors occurred for each different number of estimators set in n_estimators.
# Specifically, for n_estimators = 40 we have 24,906 errors, for n_estimators = 20, we have 26,069 errors, and for n_estimators = 50 we have 24,494 errors. We haven't tested more due to long execution times.
# In this code, the "fallos" list contains the model's predictions, while the "respuesta" list contains the correct answers. Obviously, their lengths match.


In [ ]:
qso_to_gal = []
qso_to_star = []
gal_to_star = []
star_to_gal = []
star_to_qso = []
gal_to_qso = []

for index, item in enumerate(y_pred_mlp):
    if item == 'GALAXY' and y_validation[index] == 'QSO':
        qso_to_gal.append(item)
    elif item == 'STAR' and y_validation[index] == 'QSO':
        qso_to_star.append(item)
    elif item == 'GALAXY' and y_validation[index] == 'STAR':
        star_to_gal.append(item)
    elif item == 'STAR' and y_validation[index] == 'GALAXY':
        gal_to_star.append(item)
    elif item == 'QSO' and y_validation[index] == 'STAR':
        star_to_qso.append(item)
    elif item == 'QSO' and y_validation[index] == 'GALAXY':
        gal_to_qso.append(item)

print(len(qso_to_gal), "qso to gal")
print(len(qso_to_star), "qso to star")
print(len(gal_to_star), "gal to star")
print(len(star_to_gal), "star to gal")
print(len(star_to_qso), "star to qso")
print(len(gal_to_qso), "gal to qso")

In [ ]:
# Here we will calculate the metrics for purity, completeness, F1-score, confusion matrix, and MCC:


# In case we only want to study the quasar case:
# y_validation = quas_data_sgd
# y_pred = y_pred_rf

# Transform a vector with 3 different types of elements into a vector with 2 classes to use 'binary'.
# Convert to binary: QSO or NOT QSO, with QSO as the positive class

y_validation = ['QSO' if val == 'QSO' else 'NOT_QSO' for val in y_validation]
y_pred = ['QSO' if val == 'QSO' else 'NOT_QSO' for val in y_pred_xn]


####################################
#y_pred=quas_predict_brf
#y_validation=quas_data_brf

#y_pred=y_pred_sgd


# Precision:
from sklearn.metrics import precision_score # Import the precision_score function from sklearn which directly calculates purity.
# It computes the weighted average of all individual precisions according to the number of samples in each class. 
# If we want precision per class, we could use None instead of 'weighted'.

purity = precision_score(y_validation, y_pred, average='binary', pos_label='QSO') # Compare the purity between the two lists we want.
print(f"Puresa (precisió): {purity:.4f}") # Print the purity with 4 decimals.
# Purity is the fraction of samples predicted positive as quasars among all samples predicted positive (including false positives).
# Purity = True Positives / (True Positives + False Positives)



# Recall:
from sklearn.metrics import recall_score

completitud = recall_score(y_validation, y_pred, average='binary', pos_label='QSO') #average='binary', pos_label='QSO'
print(f"Completitud: {completitud:.4f}")
# Completeness is the proportion of samples of a class that our method correctly identifies, 
# i.e., the number of true positives divided by the sum of true positives and false negatives.
# Completeness = True Positives / (True Positives + False Negatives)



# F1-score:
from sklearn.metrics import f1_score

f1 = f1_score(y_validation, y_pred, average='binary', pos_label='QSO') # Use average='macro' when we have 3 classes (y_pred and y_validation)
print(f"F1-score: {f1:.4f}")
# The F1-score ranges from 0 to 1 and measures the balance between purity (precision), i.e., false positives, and completeness, which are false negatives. If F1=1, the classification is perfect.


# Confussion matrix:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_validation, y_pred)
print("Matriu de confusió:")
print(conf_matrix)
# The confusion matrix allows us to easily visualize true positives, false positives, true negatives, and false negatives.


# We will also calculate the MCC (Matthews Correlation Coefficient), which is a measure that indicates how well a classifier performs, 
# taking into account all possibilities of both correct predictions (true positives and true negatives) and errors (false positives and false negatives).
# This metric is very good when we have a significant imbalance in the number of samples between classes, 
# whereas the F1 score can be misleading and would not perform as well.from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(y_validation, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)

# With a random forest of 50 decision trees, we obtained the following results:
# Precision: 0.9665;    Recall: 0.9669;    F1-score: 0.9646;    Confusion Matrix: [489464 1757 1646; 15860 24871 67; 5084 80 200591];    MCC: 0.929639494307183;
# We can consider the results to be very good.









In [ ]:
#split: 300, leaf: 250
'''
y_pred_rf = clf.predict(X_validation)  # Dona la predicció de cada fila d'input, és a dir, de galàxia o quàsar o estrella. És un conjunt d'etiquetes de "Galaxy", "Star", "QSO"
probs_pred = clf.predict_proba(X_validation)  # Dona les probabilitats de les prediccions per cada tipus.
qso_index = list(clf.classes_).index("QSO")  # Índex de la classe "QSO"

# Extreu les probabilitats de ser QSO
y_probs = probs_pred[:, qso_index]

# Prediccions binàries: 1 si la probabilitat > 0.8, 0 si no
y_pred = (y_probs > 0.8).astype(int)
'''



#split: 300, leaf: 100
'''
Puresa (precisió): 0.4123
Completitud: 0.8113
F1-score: 0.5467
Matriu de confusió:
[[651024  47158]
 [  7697  33085]]
Matthews Correlation Coefficient (MCC): 0.5458510722300596
'''





#split: 200, leaf: 50
'''"JPAS_DATA_Aper_Cor_3_FLUX+NOISE (1).npy"Puresa (precisió): 0.4402
Completitud: 0.8080
F1-score: 0.5700
Matriu de confusió:
[[656284  41898]
 [  7829  32953]]
Matthews Correlation Coefficient (MCC): 0.5661229836692495
'''

#split: 1000, leaf: 200
'''
Puresa (precisió): 0.3734
Completitud: 0.8188
F1-score: 0.5129
Matriu de confusió:
[[642156  56026]
 [  7391  33391]]
Matthews Correlation Coefficient (MCC): 0.5170910203348833
'''










#None:
'''
Puresa (precisió): 0.7011
Completitud: 0.5973
F1-score: 0.6450
Matriu de confusió:
[[687795  10387]
 [ 16424  24358]]
Matthews Correlation Coefficient (MCC): 0.6282565324524364
'''"JPAS_DATA_Aper_Cor_3_FLUX+NOISE (1).npy"

#40:No és millor que none
#30: No és millor que none

#50 arbres, class=None, depth=None, f1-score=0.67...

#50 arbres, class='balanced', depth=None
'''"JPAS_DATA_Aper_Cor_3_FLUX+NOISE (1).npy"Puresa (precisió): 0.7778
Completitud: 0.6083
F1-score: 0.6827
Matriu de confusió:
[[691096   7086]
 [ 15975  24807]]
Matthews Correlation Coefficient (MCC): 0.6721059900351671


In [ ]:
funcio_qso_scoring(sgdc, X_validation, y_validation)

In [ ]:
y_pred=y_pred_rf_real

#Primer de tot, per la PURESA:
from sklearn.metrics import precision_score 
purity = precision_score(y_real, y_pred, average='weighted') 
print(f"Puresa (precisió): {purity:.4f}") 


#Per la COMPLETITUD:
from sklearn.metrics import recall_score

completitud = recall_score(y_real, y_pred, average='weighted') 
print(f"Completitud: {completitud:.4f}")


#Per la F1-score:
from sklearn.metrics import f1_score
f1 = f1_score(y_real, y_pred, average='weighted') 
print(f"F1-score: {f1:.4f}")


#Per la MATRIU DE CONFUSIÓ:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_real, y_pred)
print("Matriu de confusió:")
print(conf_matrix)


#També calculadrem la MCC (Matthews Correlation Coefficient), que és una mesura que indica com de bé funciona un classificador tenint en compte totes les possibilitats tant d'encerts (veritables positius i veritables negatuis) i errors (falsos positius i falsos negatius).
#Aquesta mètrica és molt bona quan tenim nombres de mostres entre classes molt desequilibrades, mentre que la F1 es deixa enganyar i no funcionaria tan bé.
from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(y_real, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)